# Исследование надёжности заёмщиков

По заказу кредитного отдела банка, выяснить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 
Исходные данные — статистика о платёжеспособности клиентов.

Описание данных:
---
 - children — количество детей в семье
 - days_employed — общий трудовой стаж в днях
 - dob_years — возраст клиента в годах
 - education — уровень образования клиента
 - education_id — идентификатор уровня образования
 - family_status — семейное положение
 - family_status_id — идентификатор семейного положения
 - gender — пол клиента
 - income_type — тип занятости
 - debt — имел ли задолженность по возврату кредитов
 - total_income — ежемесячный доход
 - purpose — цель получения кредита

План исследования:
---
1. Изучить входные данные
2. Обработать пропуски в данных
3. Провести замену типа данных
4. Обработать дубликаты в данных
5. Провести лемматизацию
6. Провести категоризацию данных
7. Ответить на вопросы: 
<br>
-Есть ли зависимость между наличием детей и возвратом кредита в срок?
<br>
-Есть ли зависимость между семейным положением и возвратом кредита в срок?
<br>
-Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
<br>
-Как разные цели кредита влияют на его возврат в срок?
<br>

8. Предоставить общий вывод
     

### Шаг 1. Изучение общей информации. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
clients_info = pd.read_csv("/datasets/data.csv")
clients_info.info()
clients_info.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15674,20,-5419.859866,33,среднее,1,гражданский брак,1,F,сотрудник,0,155567.570284,свадьба
2300,0,376370.676855,58,среднее,1,женат / замужем,0,F,пенсионер,0,87320.676463,свой автомобиль
19465,0,-1802.397660,47,среднее,1,Не женат / не замужем,4,F,сотрудник,0,130990.217446,получение образования
20251,1,-1238.216684,38,неоконченное высшее,2,женат / замужем,0,M,сотрудник,0,360827.716031,приобретение автомобиля
16968,0,-1238.560080,52,среднее,1,женат / замужем,0,F,компаньон,0,351157.460749,покупка жилой недвижимости
10953,0,-9713.468142,44,СРЕДНЕЕ,1,женат / замужем,0,F,сотрудник,0,114889.895365,заняться высшим образованием
10889,1,NaN,26,среднее,1,Не женат / не замужем,4,F,компаньон,0,NaN,профильное образование
19203,2,-1683.106599,29,среднее,1,женат / замужем,0,F,госслужащий,0,71200.773453,строительство собственной недвижимости
19308,0,-1943.584817,41,среднее,1,гражданский брак,1,F,сотрудник,0,51190.813477,сыграть свадьбу
20326,0,-458.706771,27,высшее,0,в разводе,3,F,сотрудник,1,97872.602573,заняться высшим образованием


### Вывод

В таблице 12 столбцов. В некоторых столбцах количество значений меньше, чем в других, следовательно, в этих столбцах точно есть пропущенные значения: days_employed, total_income. Так как пропуска только в двух столбцах и их количество идентично, то, скорее всего, это не случайность.
<br>

После печати таблицы видно, что с некоторыми столбцами есть проблемы.
А именно:
отрицательный стаж в столбце days_employed
одно и тоже значение в разном написании в столбце education
в столбце purpose одни и те же цели по смыслу записаны разными словами (например, "заняться образованием" и "образование")

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
income_type_values = clients_info['income_type'].unique()

for i in income_type_values:
    median_income = clients_info[clients_info['income_type'] == i]['total_income'].median() 
    median_age = clients_info[clients_info['income_type'] == i]['days_employed'].median()
    clients_info.loc[clients_info['income_type'] == i, 'total_income'] = clients_info.loc[clients_info['income_type'] == i, 'total_income'].fillna(median_income)
    clients_info.loc[clients_info['income_type'] == i, 'days_employed'] = clients_info.loc[clients_info['income_type'] == i, 'days_employed'].fillna(median_age)

clients_info = clients_info[clients_info['dob_years'] != 0]

clients_info['children'].describe()
clients_info['children'].value_counts()
clients_info = clients_info[clients_info['children'] != 20]
clients_info['children'].value_counts()

clients_info.loc[clients_info['income_type'] == 'пенсионер', 'days_employed'] = 14580                                    
clients_info[clients_info['income_type'] == 'пенсионер']['days_employed'].describe()
clients_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21349 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21349 non-null  int64  
 1   days_employed     21349 non-null  float64
 2   dob_years         21349 non-null  int64  
 3   education         21349 non-null  object 
 4   education_id      21349 non-null  int64  
 5   family_status     21349 non-null  object 
 6   family_status_id  21349 non-null  int64  
 7   gender            21349 non-null  object 
 8   income_type       21349 non-null  object 
 9   debt              21349 non-null  int64  
 10  total_income      21349 non-null  float64
 11  purpose           21349 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Вывод

В колонках days_employed и total_income есть пропущенные значения, т.к. в пропуски по людям, у которых статус не "безработный", то скорее всего данные не передались, либо произошла какая-то ошибка. В категориях "в декрете", "студент", "безработный" пропусков нет. В остальных категориях есть пропуски. Для заполнения пропущенных значений колонки total_income, я возьму медианные значения общего дохода по категориям. Для удобства чтения данных переведу их в целочисленный формат.
Для заполнения пустых значений в колонках days_employed я беру средние значения по этой колонке для каждой категории.

101 человек с возрастом 0, это меньше 1% от общего числа строк, поэтому их можно удалить.

C помощью метода describe я обнаружила Очень большое значение в колонке дети - 20. Это значение сильно больше среднего. Т.к. таких значений всего 75, то ими можно пожертвовать и удалить из наших данных, т.к. не понятно на что их можно заменить.

Бросается в глаза очень большой стаж у категории пенсионеры. Медиана по возрасту 60 лет, максимальный возраст 74 года (если человек работал с 18 до 74 - это 20440 дне стажа). Но скорее всего, средний стаж меньше, если предположить, что рабочие года с 20 до 60 лет - получается 14 580 дней стажа. Я заменю все значения стажа для пенсионеров на 14 580.


Проверим изменения с помощью метода info()

### Замена типа данных

In [4]:
clients_info['days_employed'] = clients_info['days_employed'].astype('int').abs()
clients_info['total_income'] = clients_info['total_income'].astype('int')
clients_info['children'] = clients_info['children'].abs()
print(clients_info['children'].value_counts())
clients_info.info()

0    14080
1     4849
2     2042
3      328
4       41
5        9
Name: children, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21349 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21349 non-null  int64 
 1   days_employed     21349 non-null  int64 
 2   dob_years         21349 non-null  int64 
 3   education         21349 non-null  object
 4   education_id      21349 non-null  int64 
 5   family_status     21349 non-null  object
 6   family_status_id  21349 non-null  int64 
 7   gender            21349 non-null  object
 8   income_type       21349 non-null  object
 9   debt              21349 non-null  int64 
 10  total_income      21349 non-null  int64 
 11  purpose           21349 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

Отрицательные значения стажа переводим в положительные, т.к. стаж не моет быть отрицательный. Скорее всего была ошибка при вводе данных. Поэтому я применю метод abs() ко всему столбцу. Для удобства чтения данных перевожу значения столбцов total_income и days_employed в целочисленный формат.

В столбце дети есть отрицательное значение -1. Скорее всего это было тире и оно при обработке стало минусом, поэтому к столбцу дети я также применю метод abs().


### Обработка дубликатов

In [5]:
clients_info['education'] = clients_info['education'].str.lower()

clients_info = clients_info.drop_duplicates().reset_index(drop=True)
clients_info['purpose'].value_counts()

свадьба                                   785
на проведение свадьбы                     760
сыграть свадьбу                           756
операции с недвижимостью                  671
покупка коммерческой недвижимости         655
покупка жилья для сдачи                   648
операции с коммерческой недвижимостью     644
операции с жильем                         642
покупка жилья                             637
жилье                                     636
покупка жилья для семьи                   636
недвижимость                              628
строительство собственной недвижимости    627
операции со своей недвижимостью           626
строительство жилой недвижимости          620
покупка своего жилья                      619
строительство недвижимости                619
покупка недвижимости                      616
ремонт жилью                              603
покупка жилой недвижимости                601
на покупку своего автомобиля              502
заняться высшим образованием      

### Вывод

В столбце education много повторяющихся данных, я изменю их написание на нижний регистр.

Удаляю полные строки-дубликаты с помощью метода drop_duplicates, т.к. они вызывают смещение финальных результатов.

### Лемматизация

In [6]:
from pymystem3 import Mystem
m = Mystem()
lemmas = []
for row in range(len(clients_info)):
    lemmas.extend(m.lemmatize(clients_info.loc[row, 'purpose']))
from collections import Counter
print(Counter(lemmas))

Counter({' ': 33313, '\n': 21278, 'недвижимость': 6307, 'покупка': 5852, 'жилье': 4421, 'автомобиль': 4269, 'образование': 3980, 'с': 2891, 'операция': 2583, 'свадьба': 2301, 'свой': 2219, 'на': 2200, 'строительство': 1866, 'высокий': 1360, 'получение': 1306, 'коммерческий': 1299, 'для': 1284, 'жилой': 1221, 'сделка': 934, 'заниматься': 900, 'дополнительный': 898, 'проведение': 760, 'сыграть': 756, 'сдача': 648, 'семья': 636, 'собственный': 627, 'со': 626, 'ремонт': 603, 'подержанный': 480, 'подержать': 472, 'приобретение': 459, 'профильный': 433})


### Вывод

В столбце purpose много разных значений и некоторые из них обозначают одну и ту же цель, но написаны разными словами. Я проведу лемматизацию колонки purpose для понимания, какие слова встречаются чаще всего. Для этого я пройду циклом по всем строкам и проведу лемматизацию цели. Все значения добавлю в один лист lemmas и применю к нему Counter. 
<br>

Для дальнейшей работы я выберу наиболее популярные леммы (без учёта пробелов, спец символов, предлогов и общих слов: "покупка", "операция", "свой". Также среди двух слов синоним я выбирала наиболее популярное (например, "недвижимость" вместо "жильё") В итоге, у меня получился следующий список лемм: недвижимость, автомобиль, образование, свадьба, ремонт, прочее. Категория "прочее" добавлена на всякий случай, если в цели кредита не окажется ни одной из лемм.


### Категоризация данных

In [7]:
def category_purpose(purpose):
    if ('недвижимость' in m.lemmatize(purpose)) or ('жилье' in m.lemmatize(purpose)):
        return 'недвижимость'
    if 'автомобиль' in m.lemmatize(purpose):
        return 'автомобиль'
    if 'образование' in m.lemmatize(purpose):
        return 'образование'
    if 'свадьба' in  m.lemmatize(purpose):
        return 'свадьба'
    if 'ремонт' in m.lemmatize(purpose):
        return 'ремонт'
    else:
        return 'прочее'
    
clients_info['category_purpose'] = clients_info['purpose'].apply(category_purpose)

clients_info['income_cat'] = pd.cut(clients_info['total_income'],[0,120000,150000,300000,np.inf])

clients_info.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category_purpose,income_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,"(150000.0, 300000.0]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,"(0.0, 120000.0]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,"(120000.0, 150000.0]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,"(150000.0, 300000.0]"
4,0,14580,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,"(150000.0, 300000.0]"
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,"(150000.0, 300000.0]"
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,"(150000.0, 300000.0]"
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,"(120000.0, 150000.0]"
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,"(0.0, 120000.0]"
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,"(120000.0, 150000.0]"


### Вывод

Чтобы свести цели кредита из колонки к более общим категориям, я напишу функцию category_purpose. Функция создаёт новый столбец category_purpose в котором указана укрупненная категория (недвижимость, автомобиль, образование, свадьба, ремонт, прочее), полученные в ходе лемматизации.

C помощью метода pd.cut проведу категоризацию по доходу. Категории устанавливаю, чтобы группы примерно были равны по кол-ву человек, кроме последней, т.к. это категория людей с очень болььшим доходом, и их не может быть много.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
(clients_info.groupby('children')['debt'].agg(['count', 'mean']).sort_values('mean',ascending=False).style.format({
    'mean': '{:,.1%}'.format,}).applymap(lambda x: 'background-color : red' if x>0.3 else '', subset=['mean']))

,count,mean
children,,
4,41,9.8%
2,2039,9.5%
1,4839,9.1%
3,328,8.2%
0,14022,7.5%
5,9,0.0%


### Вывод

Для ответа на этот вопрос я сгруппирую таблицу с количеством детей, в качестве значений беру колонку debt и считаю кол-во и среднюю по ней. 

Из таблицы видно, что у людей с 5 детьми, нет долгов и они являютя надёжными заёмщиками. Но выборка очень мала - всего 9 наблюдений, хотя, наверное, статистически таких семей неочень много. При этом люди с 4 детьми являются "худшими" заёмщиками, поэтому сделать вывод, что с ростом количества детей благонадежность заёмщика повышается - нельзя.

Но, в среднем, те, у кого нет детей отдают долги лучше тех, у кого они есть.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
(clients_info.groupby('family_status')['debt'].agg(['count', 'mean']).sort_values('mean',ascending=False).style.format({
    'mean': '{:,.1%}'.format,}).applymap(lambda x: 'background-color : red' if x>0.3 else '', subset=['mean']))

,count,mean
family_status,,
Не женат / не замужем,2785,9.8%
гражданский брак,4118,9.3%
женат / замужем,12242,7.5%
в разводе,1183,7.1%
вдовец / вдова,950,6.5%


### Вывод

Для ответа на этот вопрос я сгруппирую таблицу по семейному положению, в качестве значений беру колонку debt и считаю кол-во и среднюю по ней. 
Хуже всего отдают долг не женатые/не замужние и те, кто живёт в гражданском браке. Можно сделать первичный вывод, что эти люди менее ответственные, по сравнению с теми, кто состоит или состоял в браке. Также, возможно, что эти люди моложе остальных категорий.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
clients_info['income_cat'] = pd.cut(clients_info['total_income'],[0,100000,200000,300000,500000,np.inf])
(clients_info.groupby('income_cat')['debt'].agg(['count', 'mean']).sort_values('income_cat',ascending=True).style.format({
    'mean': '{:,.1%}'.format,}).applymap(lambda x: 'background-color : red' if x>0.3 else '', subset=['mean']))

,count,mean
income_cat,,
"(0.0, 100000.0]",4429,8.0%
"(100000.0, 200000.0]",11828,8.6%
"(200000.0, 300000.0]",3547,7.0%
"(300000.0, 500000.0]",1252,7.3%
"(500000.0, inf]",222,6.3%



### Вывод

Для ответа на этот вопрос я сделаю новый столбец income_cat и сделаю разбивку по группам дохода. Далее я сгруппирую таблицу по категории дохода, в качестве значений беру колонку debt и считаю кол-во и среднюю по ней. 
По результатам видно, что люди с очень большим доходом чаще отдают долг вовремя. И, в среднем, люди с доходом более 200 тыс. отдают долг чаще. Значит, зависимость между уровнем дохода и возвратом кредита в срок есть.

- Как разные цели кредита влияют на его возврат в срок?

In [11]:
clients_info.pivot_table(index = 'category_purpose', values = ['debt'], aggfunc = ['count', 'mean'])

,count,mean
,debt,debt
category_purpose,,
автомобиль,4269,0.093230
недвижимость,10728,0.072427
образование,3980,0.092714
свадьба,2301,0.078661


### Вывод

Для ответа на этот вопрос я сделаю сводную таблицу с помощью метода pivot_table. В качестве значений строк я возьму цели кредита, полученные в ходе лемматизации, в качестве значений беру колонку debt и считаю кол-во и среднюю по ней. 
Лучше всего отдают долг те, кто берёт кредит на недвижимость, а хуже, те, кто берёт кредит на автомобиль.

### Шаг 4. Общий вывод

В данной работе я обработала данные от пропусков, удалила дубликаты и ошибки.
Я провела лемматизацию и сделала на её основе категоризацию по целям кредита.
Также я провела категоризацию по доходу.

С помощью методов сводных таблиц и группировки данных был получен ответ на следующие вопросы:

- Есть ли зависимость между наличием детей и возвратом кредита в срок?
<br>**Да, зависимость есть. В среднем, те, у кого нет детей отдают долги лучше тех, у кого они есть.**
<br>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?
<br>**Да, зависимость есть. Хуже всего отдают долг не женатые/не замужние и те, кто живёт в гражданском браке.**
<br>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
<br>**Да, зависимость есть. В среднем, люди с доходом более 200 тыс. более благонадёжные.**
<br>

- Как разные цели кредита влияют на его возврат в срок?
<br>**Лучше всего отдают долг те, кто берёт кредит на недвижимость, а хуже, те, кто берёт кредит на автомобиль.**
<br>

Можно сделать вывод, что люди без детей, вдовцы или разведённые, с доходом более 200 тыс и берущие кредит на недвижимость отдают долг лучше остальных. 

При этом в данных были ошибки и неточности (очень большой стаж у пенсионеров, 20 детей, отрицательные значения стажа и детей), нужно выявить причину их появления и постараться устранить её в будущем.